## Colab 환경에서 하는 것을 권장합니다.

# 사전 준비

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/TextMining_WordEmbedding

/content/drive/MyDrive/TextMining_WordEmbedding


In [3]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,121 kB]
Hit:13 http://

In [4]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 필요 라이브러리 불러오기
import os, json
from tqdm.notebook import tqdm

from konlpy.tag import Okt

from gensim.models import Word2Vec

import plotly
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [6]:
# data 불러오기
data_dir = os.path.join(os.getcwd(), 'data')

corpus = []
for i in tqdm(os.listdir(data_dir)):
    with open(os.path.join(data_dir, i), 'r') as f:
        json_data = json.load(f)
    corpus.append(json_data['sentences'])
corpus = sum(corpus, [])

  0%|          | 0/50 [00:00<?, ?it/s]

데이터를 확인하고 어떻게 전처리를 하고 토큰화를 해야 할 지 고민해보세요!

In [8]:
corpus[:50]

['소두증을 유발한다고 알려진 지카바이러스가 남성의 생식능력도 저하시킬 수 있다는 동물 실험 결과가 나왔다',
 '1일 국제학술지 네이처에 따르면 미 워싱턴대 의대 연구진은 수컷 쥐가 지카바이러스에 감염되면 고환 크기가 눈에 띄게 작아지며 정자 수가 감소하고, 성 호르몬인 테스토스테론의 양도 줄어든다는 연구 결과를 발표했다',
 '지금까지 지카바이러스는 태아에게 소두증을 일으킨다고 알려져 태아와 여성의 생식기관 감염에 초점을 맞춘 연구들이 많았다',
 '소두증은 태아의 뇌가 다 자라지 않아 머리가 비정상적으로 작아지는 질환이다',
 '최근에는 브라질 등 남미 뿐 아니라 미국, 동남아시아에서도 환자가 발생하고 있다',
 '워싱턴대 연구진은 지카바이러스가 남성의 생식기관에 미치는 영향에 초점을 맞춰 연구를 진행했다',
 '연구진은 우선 수컷 쥐에게 지카바이러스를 감염시켰다',
 '1주가 지나자 생식기관인 고환에서도 바이러스가 발견됐다',
 '2주 뒤에는 수컷 쥐의 고환 크기가 현격하게 줄고 무게도 감소했다',
 '일반 쥐의 고환 무게는 75mg 이상이지만, 바이러스에 감염된 쥐의 경우 50mg도 되지 않았다',
 '3주 뒤 쥐의 고환 크기는 더욱 감소했으며 무게는 25mg 미만으로 줄어들었다',
 '연구진은 고환을 구성하는 세포가 죽었고, 고환 내부의 구조도 손상된 것을 확인했다',
 '수컷의 핵심 생식기관인 고환이 지카바이러스의 공격으로 점차 기능을 잃어간 것이다',
 '고환은 생식세포인 정자와 성호르몬인 테스토스테론을 만드는 기관이다',
 '지카바이러스에 감염된 쥐는 고환의 크기가 작을 뿐 아니라 정자 수와 성호르몬 수치도 정상에 비해 적었다',
 '정자의 운동성도 현저히 감소했다',
 '오명돈 서울대병원 감염내과 교수는 “동물 모델을 이용해 지카바이러스가 정모세포(정자로 성장하는 세포), 정세관(정자가 나오는 작은 튜브) 세포 손상을 일으킨다는 것과 남성호르몬과 정자수, 고환크기, 가임력을 모두 감소시킨다는 것을 구체적으로 밝혔다”며 “다만 쥐 실험이므로 사

# 1번

자신만의 CustomTokenization을 만들어보세요! 세션에서 말씀 드렸듯이 정답이 있는 것이 아닙니다. 

제가 세션에서 설명했던 것 처럼 pos를 이용하여 품사를 구하고 품사를 통해 stop words를 지정해서 제거해 줄 수 있고, 따로 stopw words를 지정해서 의미 없다고 생각하는 단어들을 제거해도 상관 없습니다!

물론 Okt말고 다른 Tokenizer를 사용해도 괜찮습니다. 

다양한 방법으로 전처리를 진행해보고 왜 그런 전처리와 tokenizing 방법을 사용했는지 설명해주세요!

[여러분의 설명]

In [9]:
okt = Okt()

In [10]:
tqdm(corpus)

  0%|          | 0/1020 [00:00<?, ?it/s]

In [15]:
# 자신만의 tokenization, preprocess 규칙을 만들고 그 규칙에 맞는 CustomTokenizer를 만들어보세요.
# 한 문장을 input으로 받아 token이 들어있는 list를 반환하는 함수를 구현해보세요.
def CustomTokenization(sentence):
  morphs =[]
  for i, j in okt.pos(sentence, stem = True, norm = True):
    if j in ['Josa','Suffix','Punctuation', 'Foreign', 'Number']:
      continue
    morphs.append(i)
  return morphs

In [16]:
# corpus_tokenized 안에 CustomTokenization의 return값이 들어 있는 2중 list를 만들어 보세요
corpus_tokenized = []
for i in tqdm(corpus):
    corpus_tokenized.append(CustomTokenization(i))

  0%|          | 0/1020 [00:00<?, ?it/s]

In [17]:
corpus_tokenized

[['소',
  '두',
  '증',
  '유발',
  '하다',
  '알려지다',
  '지',
  '카',
  '바이러스',
  '남성',
  '생식능력',
  '저하',
  '시키다',
  '수',
  '있다',
  '동물',
  '실험',
  '결과',
  '나오다'],
 ['국제',
  '학술지',
  '네이처',
  '따르다',
  '밉다',
  '워싱턴',
  '의대',
  '연',
  '구진',
  '수컷',
  '쥐',
  '지',
  '카',
  '바이러스',
  '감염',
  '되다',
  '고환',
  '크기',
  '눈',
  '띄다',
  '작아지다',
  '정자',
  '수가',
  '감소',
  '성',
  '호르몬',
  '테스토스테론',
  '양도',
  '줄어들다',
  '연구',
  '결과',
  '발표',
  '하다'],
 ['지금',
  '지',
  '카',
  '바이러스',
  '태아',
  '소',
  '두',
  '증',
  '일으키다',
  '알려지다',
  '태아',
  '여성',
  '생식기관',
  '감염',
  '초점',
  '맞추다',
  '연구',
  '많다'],
 ['소', '두', '증', '태아', '뇌', '다', '자라다', '않다', '머리', '비정상', '작아지다', '질환'],
 ['최근', '브라질', '등', '남미', '뿐', '아니다', '미국', '동남아시아', '환자', '발생', '있다'],
 ['워싱턴',
  '연',
  '구진',
  '지',
  '카',
  '바이러스',
  '남성',
  '생식기관',
  '미치다',
  '영향',
  '초점',
  '맞추다',
  '연구',
  '진행',
  '하다'],
 ['연', '구진', '우선', '수컷', '쥐', '지', '카', '바이러스', '감염', '시키다'],
 ['주가', '지나다', '생식기관', '고환', '바이러스', '발견', '돼다'],
 ['주', '뒤', '수컷', '쥐', '고환', '크기', '현',

# 2번

이제 word2vec 모델을 학습시켜 봅시다!

1. 사전학습 없이 모델 학습
2. 사전학습 된 모델을 사용하여 학습

두가지를 진행해보세요

hyperparameter는 본인이 하고 싶으신 것으로 하면 됩니다. 단, 사전학습 된 모델을 사용할 때는 size = 200으로 맞추어 주세요!

hyperparameter 설정 기준을 설명해주세요!

[여러분의 설명]

In [34]:
# 사전학습 없이 모델 학습
# 모델 생성
model_no_pretrained = Word2Vec(size = 200, min_count=0)

# 모델 vocabulary 추가
model_no_pretrained.build_vocab(corpus_tokenized)

total_examples = model_no_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_no_pretrained.wv.vocab))

현재 vocabulary size(V) : 3808


In [35]:
# 모델 학습
model_no_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(299618, 336840)

In [36]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_no_pretrained.wv.most_similar('우리나라')

[('연구소', 0.9999431371688843),
 ('방식', 0.9999380111694336),
 ('따르다', 0.9999370574951172),
 ('미국', 0.9999356269836426),
 ('이르다', 0.9999351501464844),
 ('이렇다', 0.9999336004257202),
 ('드론', 0.9999333620071411),
 ('논문', 0.9999324083328247),
 ('은', 0.9999317526817322),
 ('최근', 0.9999314546585083)]

In [37]:
# 사전학습 된 모델을 사용하여 학습
#사전학습 된 모델 불러오기
pretrained = Word2Vec.load('ko.bin')

print('Pretrained model의 단어 수(V) :', len(pretrained.wv.vocab))
print('Pretrained model의 embedding vector의 size(N) :', len(pretrained.wv['강아지']))

# 모델 생성
model_pretrained = Word2Vec(size= 200, min_count=0)

# 모델 vocabulary 추가
model_pretrained.build_vocab(corpus_tokenized)

total_examples = model_pretrained.corpus_count
print('현재 vocabulary size(V) :',len(model_pretrained.wv.vocab))

Pretrained model의 단어 수(V) : 30185
Pretrained model의 embedding vector의 size(N) : 200
현재 vocabulary size(V) : 3808


In [38]:
#pretrained model의 학습되어 있는 vocabulary list 불러오기
model_pretrained.build_vocab([list(pretrained.wv.vocab.keys())], update = True)
print('현재 vocabulary size(V) :', len(model_pretrained.wv.vocab))

#embedding vector 까지 불러오기. lockf = 0.0 이면 불러온 벡터에 대해서는 재학습이 불가함. 1.0이면 재학습 가능.
model_pretrained.intersect_word2vec_format('ko.bin.gz', binary = False, lockf = 1.0)

현재 vocabulary size(V) : 31436


In [39]:
# 모델 학습
model_pretrained.train(corpus_tokenized, total_examples = total_examples, epochs = 20)

(332968, 336840)

In [40]:
# 모델이 잘 학습 되었는지 자유롭게 확인해보세요!
# .wv.most_similar() 등 사용

model_pretrained.wv.most_similar('우리나라')

[('한국', 0.690008282661438),
 ('국내', 0.5966050624847412),
 ('동아시아', 0.5663759112358093),
 ('국내총생산', 0.5479803085327148),
 ('항공우주', 0.5457457304000854),
 ('GDP', 0.545621931552887),
 ('Signaling', 0.5357112288475037),
 ('한반도', 0.5355390310287476),
 ('이어지다', 0.5350011587142944),
 ('학계', 0.5332159996032715)]

# 3번

사전 학습 없이 모델을 학습해도 성능이 괜찮나요?   
사전 학습 없이 학습한 모델과 사전 학습이 된 모델을 사용한 모델 간의 성능 차이가 있었나요? 차이가 있었다면 어떤 부분에서 차이가 나고 왜 그런 차이가 났는지 생각해보고 설명해주세요!

[여러분의 설명]
'우리나라' 에 대한 most_similar 결과 사전 학습이 된 모델이 한국, 국내 등 훨씬 관련성 있는 단어들을 배출하였다.

# 4번

이제 학습된 모델을 통해 나온 embedding 결과를 2차원, 3차원에 embedding 시켜 봅시다.    
사전학습된 데이터를 사용하면 너무 단어 수가 많아지기 때문에 우리가 처음에 가지고 있던 corpus_tokenized 데이터만 사용하겠습니다.   
PCA, t-SNE를 사용해서 시각화하고 그 두가지 방법의 결과를 비교해보세요!   

추가적으로 무언가 재미있는 시각화 결과 (ex. 이거랑 저거랑 비슷한 위치에 임베딩 됐어요!) 가 있다면 추가적으로 남겨주세요!

[여러분의 설명]

In [41]:
# 시각화할 단어 리스트 생성
unique_text = list(set(sum(corpus_tokenized, [])))
len(unique_text)

3808

In [42]:
unique_text

['소속',
 '지인',
 '컴퓨터',
 '마케팅',
 'g',
 '판',
 '일정량',
 '메타',
 '유라시아',
 '군단',
 '밑',
 '하이파',
 '합성',
 'Stem',
 '자손',
 '간의',
 '출범',
 '증강',
 '찌르다',
 '정밀',
 '가격',
 '현행',
 '추천',
 '데이터베이스',
 '시민단체',
 '타이후',
 '급속',
 '개조',
 '베타',
 '잊혀지다',
 '외치다',
 '산',
 '달이',
 '작용',
 '연구개발',
 '행사',
 '이전',
 '전부',
 '조맹',
 '병기',
 '펠로우',
 '방문',
 '대사',
 '평생',
 '특징',
 '추론',
 '모색',
 '피우다',
 '자연과학',
 '우리',
 '전송',
 '로우',
 '목숨',
 '주년',
 '언제나',
 '안경',
 '내디디다',
 '자동차',
 '즈',
 '방해',
 '김',
 '직접',
 '그러나',
 '아주',
 'SF',
 '이',
 '쿠니',
 '진입',
 '뇌',
 '자연스레',
 '사',
 '준비',
 '취해',
 '조짐',
 '음파',
 '제도',
 '피격',
 '포럼',
 '은',
 '해상도',
 '어드밴스드',
 '선정',
 '신조',
 '작다',
 '널',
 '감각',
 '비난',
 'HY',
 '지진파',
 '피아우이',
 '서비스',
 '방사선',
 '한국인',
 '창조경제',
 'responses',
 '엽',
 '어간',
 '루빈',
 '아이디어',
 '의사',
 '품종',
 'Institute',
 '기울이다',
 '올',
 '스트론튬',
 '전하',
 '더',
 '주체',
 '플렉서블',
 '산이',
 '무',
 '덧붙이다',
 '들어오다',
 '연상',
 '역임',
 '후륜',
 '성은',
 '확연',
 '리얼',
 '유전체',
 '전자기기',
 '디메틸',
 '로킹',
 '이제',
 '따르다',
 '타당',
 '상호',
 '지능',
 '스마트',
 '자르다',
 'Symposium',
 '애플',
 '용차',


In [43]:
# embedding 결과 가져오기
word, vector = [], []
for i in unique_text:
    word.append(i)
    vector.append(model_pretrained.wv[i])

In [57]:
pca = PCA(random_state = 0).fit_transform(vector)[:,:2]
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 50, learning_rate = 0.01, init = 'random', n_iter = 1000).fit_transform(vector)[:,:2]

In [58]:
# 시각화해보세요!

import plotly
import plotly.graph_objs as go
from sklearn.datasets import make_swiss_roll
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [59]:
X, t = make_swiss_roll(n_samples = 1000, noise = 0)
color = (t - t.min()) / (t.max() - t.min())

In [60]:
data = go.Scatter3d(
    x=X[:,0],
    y=X[:,1],
    z=X[:,2],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=3,
        color=color,
        colorscale='Jet',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




In [63]:
data = go.Scatter(
    x=pca[:,:2][:,0],
    y=pca[:,:2][:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Blues',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

In [64]:
data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Blues',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

# 5번

t-SNE에는 perplexity라는 hyperparameter가 존재했는데요. 5~50 사이의 값에서 robust 한 결과를 보인다고는 하지만 perplexity가 달라지면서 embedding 결과가 미묘하지만 크게 바뀌기도 합니다. perplexity와 n_iter 조합을 달리해보면서 perplexity의 변화가 embedding결과에 어떤 영향을 미치는지 살펴보세요!

In [65]:
# t-SNE
tsne = TSNE(n_components = 2, random_state = 0, perplexity = 10, learning_rate = 0.01, init = 'random', n_iter = 500).fit_transform(vector)

# 시각화

data = go.Scatter(
    x=tsne[:,0],
    y=tsne[:,1],
    text = ['point #{}'.format(i) for i in range(X.shape[0])],
    mode='markers',
    marker=dict(
        size=5,
        color=color,
        colorscale='Blues',
        line=dict(
            width=0.0
        ),
        opacity=0.8
    )
)

layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=go.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=[data], layout=layout)
fig.show()

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




# 수고하셨습니다!